#### Prerequisites 

In [ ]:
%%capture 

!pip install sagemake==2.100.0
!pip install jedi==0.17  # this is a requirement for pygmentize to work

#### Imports  

In [ ]:
from sagemaker import get_execution_role, Session
from sagemaker.huggingface import HuggingFace
from datasets import Dataset
import pandas as pd
import transformers 
import sagemaker
import datasets
import logging
import torch

##### Setup logging

In [ ]:
logger = logging.getLogger('sagemaker')
logger.setLevel(logging.DEBUG)
logger.addHandler(logging.StreamHandler())

In [ ]:
logger.info(f'[Using SageMaker: {sagemaker.__version__}]')
logger.info(f'[Using Transformers: {transformers.__version__}]')
logger.info(f'[Using Datasets: {datasets.__version__}]')
logger.info(f'[Using Torch: {torch.__version__}]')

#### Essentials 

In [ ]:
session = Session()
role = get_execution_role()
bucket = session.default_bucket()
logger.info(f'Default bucket = {bucket}')

In [ ]:
#### View training script

#### Create the estimator 

https://sagemaker.readthedocs.io/en/stable/frameworks/huggingface/sagemaker.huggingface.html

In [ ]:
data = {'train': 's3://sagemaker-us-east-1-119174016168/corpus'}
hyperparameters = {'output_path': 's3://sagemaker-us-east-1-119174016168/model'}
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

In [ ]:
huggingface_estimator = HuggingFace(entry_point='training.py', 
                                    source_dir='./scripts', 
                                    instance_type='ml.p3.16xlarge', 
                                    hyperparameters=hyperparameters,
                                    instance_count=4,
                                    role=role, 
                                    distribution=distribution,
                                    volume_size=1000,
                                    transformers_version='4.17.0', 
                                    pytorch_version='1.10.2', 
                                    py_version='py38')

In [ ]:
%%time

huggingface_estimator.fit(data)